# Comparison of $\gamma$-PhysNet with the state-of-the-art method used in CTAO LST-1 here named _lstchain_.

The lstchain IRFs used here have been published in Abe et al 2023.

Note the following caveats:
- The datasets are different. We chose to compare with the closest ones: 23.6deg (vs 20 degrees zenith for $\gamma$-PhysNet).
- The event selection is different as Abe et al 2023 removed wrong-tail events (flattening the angular resolution at low energies for lstchain), which is not possible in our case

These differences appart, the IRFs have been computed the same way, using pyIRF and a gamma efficiency of 70\%.

Nevertheless, comparing the Vanilla version of $gamma$-PhysNet with lstchain was not the goal of the published study (and has been done in previous studies). We include it here as convenience.

In [ ]:
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
from pathlib import Path
import plot_irfs

import warnings
warnings.filterwarnings("ignore")

plt.style.use('seaborn-paper')
plt.style.use('seaborn-colorblind')

In [ ]:
efficiencies = [0.4, 0.7, 0.9]
zeniths = [10.00, 23.630, 32.06, 43.20]

erange = (15*u.GeV, 25*u.TeV)

angresylim = (0.0, 0.5)
effareaylim = (1e3, 1e6)
eneylim = (-0.16, 0.7)

opt = dict(elinewidth=0.8)

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
indir = '3_pipeline/3.4_irfs/irfs'
outdir = 'irfs_plots'
Path(outdir).mkdir(exist_ok=True)
print(list(Path(indir).iterdir()))
print(list(Path(outdir).iterdir()))

work_dir = Path.cwd().parent

In [ ]:
from girfs.irfs import IRFGammaPointSource, IRF, EnergyBias

files = list(work_dir.glob(f'irfs/vanilla/best_scenario/seed_*/irfs.fits.gz'))
irfs_gphysnet = IRFGammaPointSource.from_files(files)
ebias_gphysnet = EnergyBias.from_files(files)

In [ ]:
zen = 23.63
efficiencies = [0.7]

fig, axes = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

for ii, efficiency in enumerate(efficiencies):
    filename = Path(indir, f'irfs_zen_{zen:.2f}_gh-eff_{efficiency}.fits.gz')
    print(filename)

    plot_irfs.plot_angular_resolution_from_file(filename, ax=axes[0],
                                                ls='-',
                                                **opt
                                               )
    plot_irfs.plot_effective_area_from_file(filename, ax=axes[1],
                                            ls='-',
                                            label=f'H. Abe et al 2023 - zenith {zen:.2f}°',
                                            **opt
                                           )
    plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[2], 
                                               ls='-',
                                               color=colors[ii],
                                               **opt
                                              )
    plot_irfs.plot_energy_bias_from_file(filename, ax=axes[2],
                                         color=colors[ii],
                                         ls='dotted',
                                         **opt
                                        )
    

plot_irfs.plot_energy_resolution_from_file(filename, ax=axes[2], 
                                           ls='-',
                                           color=colors[ii],
                                           label=r'Energy resolution ($\sigma_{68}$)',
                                           **opt
                                              )
plot_irfs.plot_energy_bias_from_file(filename, ax=axes[2],
                                         color=colors[ii],
                                         ls='dotted',
                                     label=r'Energy bias (median)',
                                     **opt
                                    )


irfs_gphysnet.effective_area.plot(label=r'$\gamma$-PhysNet Vanilla - zenith 20°', ax=axes[1], color=colors[-3])
irfs_gphysnet.energy_resolution_bias.plot(ax=axes[2], color=colors[-3])
ebias_gphysnet.plot(ax=axes[2], color=colors[-3], ls='dotted')
irfs_gphysnet.angular_resolution.plot(ax=axes[0], color=colors[-3])



for ax in axes:
    ax.set_title('')
    ax.set_xlim(erange[0].to_value(u.TeV), erange[1].to_value(u.TeV))
    
    
# axes[0].set_title(f'Zenith {zen:.2f}°')
axes[0].set_ylim(0, axes[0].get_ylim()[1])
axes[0].legend().remove()
axes[0].set_xlabel('')
axes[1].set_xlabel('')
axes[0].set_ylim(*angresylim)
axes[1].set_ylim(*effareaylim)
axes[2].set_ylim(*eneylim)
axes[2].set_ylabel(r'$\Delta E/E_{true}$')
axes[2].legend()

axes[1].grid(True, which='both', axis='x')
axes[1].grid(True, which='major', axis='y')
for ax in axes:
    ax.grid(False, axis='x', which='minor')
    ax.grid(False, axis='y', which='minor')

axes[1].set_axisbelow(True)
handles, labels = axes[1].get_legend_handles_labels()
axes[1].legend(handles[1:], labels[1:], loc='lower right')


plt.tight_layout()
outfile = Path(outdir, f'irfs_eff_zen_{zen:.2f}.png')
plt.savefig(outfile, dpi=250)

These plots show the superiority of the $\gamma$-PhysNet Vanilla model over the baseline model of lstchain.    
The gammaPhysNet Vanilla model has a better angular resolution, effective area, and energy resolution and bias on the majority of the energy range, in particular at low energies.